In [5]:
import os
import numpy as np
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
from dataclasses import asdict

In [6]:
data, info = tfds.load('glue/mnli', with_info=True)

INFO:absl:Load dataset info from /aiffel/tensorflow_datasets/glue/mnli/2.0.0
INFO:absl:Reusing dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


In [7]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['premise']
    sentence2 = example['hypothesis']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [9]:
class MnliProcessor(DataProcessor):
    """Processor for the MNLI data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        # print(tensor_dict)
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy())
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(saelf._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"] # ['entailment', 'neutral', 'contradiction']

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for i, line in enumerate(lines):
            if i == 0:
                continue
            guid = f"{set_type}-{line[0]}"
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))

In [10]:
processor = MnliProcessor()

In [12]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [13]:
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 3)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    return features

In [15]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

In [16]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

Using label list ['0', '1', '2']
Using label list ['0', '1', '2']
Using label list ['0', '1', '2']


In [17]:
num_classes = len(processor.get_labels())
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_dataset_batch, epochs=20, steps_per_epoch=115, 
          validation_data=validation_dataset_batch)

Epoch 1/20
115/115 [==============================] - 152s 1s/step - loss: 1.0010 - acc: 0.4951 - val_loss: 0.8838 - val_acc: 0.6062
Epoch 2/20
115/115 [==============================] - 138s 1s/step - loss: 0.8473 - acc: 0.6185 - val_loss: 0.7657 - val_acc: 0.6673
Epoch 3/20
115/115 [==============================] - 138s 1s/step - loss: 0.7901 - acc: 0.6620 - val_loss: 0.7459 - val_acc: 0.6743
Epoch 4/20
115/115 [==============================] - 138s 1s/step - loss: 0.7317 - acc: 0.6908 - val_loss: 0.6938 - val_acc: 0.7115
Epoch 5/20
115/115 [==============================] - 138s 1s/step - loss: 0.7096 - acc: 0.7043 - val_loss: 0.6816 - val_acc: 0.7135
Epoch 6/20
115/115 [==============================] - 138s 1s/step - loss: 0.7540 - acc: 0.6772 - val_loss: 0.6486 - val_acc: 0.7346
Epoch 7/20
115/115 [==============================] - 138s 1s/step - loss: 0.6743 - acc: 0.7158 - val_loss: 0.6523 - val_acc: 0.7275
Epoch 8/20
115/115 [==============================] - 138s 1s/step - 

In [20]:
# 평가
result = model.evaluate(test_dataset_batch)
print(result)

613/613 [==============================] - 92s 150ms/step - loss: 1.9799 - acc: 0.3544
[1.9799156188964844, 0.3544303774833679]
